## Google Photos API - Download images from Google Photos using Python

Using the Google Photos REST API you can download, upload and modify images stored in Google Photos.

The following steps describe how to set up a simple project that lets you use Python to download images from Google Photos:

## Create virtualenv and install required packages

1. Open the terminal and navigate to your working directory. The folder structure of the repo includes the following directories:

    * **credentials**: folder to store the credentials you need to authenticate your "Python App" to the Google Photos Library
    * **media_items_list**: every time the script runs, I want to save a .csv file with all Google Photos media items and the corresponding metadata uploaded in the defined time period
    * **downloads**: storing downloaded images from Google Photos


2. Create a virtual environment `python3 -m venv venv`, activate it `. ./venv/bin/activate` and install requirements `pip install -r requirements.txt`

3. Install ipykernel which provides the IPython kernel for Jupyter: `pip install ipykernel` and add your virtual environment to Jupyter: `python -m ipykernel install --user --name=venv` 

    You can check the installation by navigating to /Users/<user>/Library/Jupyter/kernels. There should be a new directory called 'venv'. In the folder you can find the file 'kernel.json', which contains the path for the used python installation is defined.

4. Start jupyter notebook or jupyter lab: `jupyter lab .` and select the just created environment "venv" as Kernel

![](read_me_img/select_kernel.png)

## Enable Google API

5. Enable Google Photos API Service

   1. Go to the Google API Console [https://console.cloud.google.com/](https://console.cloud.google.com/). 
   2. From the menu bar, select a project or create a new project.
   
      ![](read_me_img/gifs/create_new_project_speed.gif)
      
   3. To open the Google API Library, from the Navigation menu, select APIs & Services > Library. 
   4. Search for "Google Photos Library API". Select the correct result and click "enable". If its already enabled, click "manage"
   
       ![](read_me_img/gifs/enable_api_speed.gif)
       
   5. Afterwards it will forward you to the "Photos API/Service details" page (https://console.cloud.google.com/apis/credentials)


6. Configure "OAuth consent screen" ([Source](https://stackoverflow.com/questions/65184355/error-403-access-denied-from-google-authentication-web-api-despite-google-acc))

   1. Go back to the Photos API Service details page and click on "[OAuth consent screen](https://console.cloud.google.com/apis/credentials/consent)" on the left side (below "Credentials") 
   2. Add a Test user: Use the email of the account you want to use for testing the API call
   
        ![](read_me_img/add_test_user.png)

7. Create API/OAuth credentials

   1. On the left side of the Google Photos API Service page, click Credentials
   2. Click on "Create Credentials" and create a OAuth client ID
   3. As application type I am choosing "Desktop app" and give your client you want to use to call the API a name
   4. Download the JSON file to the created credentials, rename it to "client_secret.json" and save it in the folder "credentials"
   
        ![](read_me_img/gifs/create_credentials_speed.gif)

## Install and import required packages

In [1]:
%%capture capt 
#saves the output to variable capt, to print output capt.stdout, capt.stderr
!pip install -r "requirements.txt"
!pip freeze > requirements.txt

In [2]:
!which python
!which pip

/Users/andyflury/opt/anaconda3/bin/python
/Users/andyflury/opt/anaconda3/bin/pip


## Use the Google Photo Library API for the first time:

The following section shows how to use OAuth Credentials for authentication with the Google Library API. The code section below covers the following steps:

8. Create a service for the first time:

    1. Initialize GooglePhotosApi `google_photos_api = GooglePhotosApi()`

    2. Create Service using the `client_secret.json` file: `service = google_photos_api.create_service()`
        
        
       <b>Calling the API for the first time:</b>
       1. Google will ask you if you want to grant the App the required permissions you defined with the scope:
       ![](read_me_img/sign_in_google_acc.png)
       2. Since its just a test app at the moment, Google will make you aware of that > Click on "Continue"
       3. Once you granted the app the required permissions, you will see a "token_......pickle" file created in the folder "credentials". This token file will be used for future calls.

In [2]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import requests

class GooglePhotosApi:
    def __init__(self,
                 api_name = 'photoslibrary',
                 client_secret_file= r'./credentials/client_secret.json',
                 api_version = 'v1',
                 scopes = ['https://www.googleapis.com/auth/photoslibrary']):
        '''
        Args:
            client_secret_file: string, location where the requested credentials are saved
            api_version: string, the version of the service
            api_name: string, name of the api e.g."docs","photoslibrary",...
            api_version: version of the api

        Return:
            service:
        '''

        self.api_name = api_name
        self.client_secret_file = client_secret_file
        self.api_version = api_version
        self.scopes = scopes
        self.cred_pickle_file = f'./credentials/token_{self.api_name}_{self.api_version}.pickle'

        self.cred = None

    def run_local_server(self):
        # is checking if there is already a pickle file with relevant credentials
        if os.path.exists(self.cred_pickle_file):
            with open(self.cred_pickle_file, 'rb') as token:
                self.cred = pickle.load(token)

        # if there is no pickle file with stored credentials, create one using google_auth_oauthlib.flow
        if not self.cred or not self.cred.valid:
            if self.cred and self.cred.expired and self.cred.refresh_token:
                self.cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.client_secret_file, self.scopes)
                self.cred = flow.run_local_server()

            with open(self.cred_pickle_file, 'wb') as token:
                pickle.dump(self.cred, token)
        
        return self.cred


## Initialize photos api and create service

In [48]:
google_photos_api = GooglePhotosApi()
creds = google_photos_api.run_local_server()

## Use pythons requests module and the token file to retrieve data from Google Photos

The functions here
1. request media items from Hilledwight shared album and store the item IDs in picIds.txt
2. request media items from all my photos organized by month and store the items IDs respective txt files in the idFiles directory

We're limiting the number of pictures requested by numPics in each function, but in the future we'll remove numPics

In [46]:
import json
import requests

PHOTOS_HEADERS = {
    'content-type': 'application/json',
    'Authorization': 'Bearer {}'.format(creds.token)
}

GET_LIBRARIES_URL = 'https://photoslibrary.googleapis.com/v1/albums'
MEDIA_ITEMS_URL = 'https://photoslibrary.googleapis.com/v1/mediaItems:search'

MAX_NUM_PICS = 39
MONTHS = [
    {'month': 8, 'year':2021, 'name':'August'}, 
    {'month': 9, 'year':2021, 'name': 'September'}, 
    {'month': 10, 'year':2021, 'name': 'October'}, 
    {'month': 11, 'year':2021, 'name': 'November'}, 
    {'month': 12, 'year':2021, 'name': 'December'}, 
    {'month': 1, 'year':2022, 'name': 'January'}, 
    {'month': 2, 'year':2022, 'name': 'February'}, 
    {'month': 3, 'year':2022, 'name': 'March'},
    {'month': 4, 'year':2022, 'name': 'April'}, 
    {'month': 5, 'year':2022, 'name': 'May'}, 
    {'month': 6, 'year':2022, 'name': 'June'}, 
]

CATEGORIES = [
    ['ANIMALS,PETS','ANIMALS'],
    ['FASHION'],
    ['LANDMARKS'],
    ['ARTS'],
    ['FLOWERS','GARDENS','LANDSCAPES', 'NATURE'],
    ['BIRTHDAYS'],
    ['FOOD'],
    ['NIGHT'],
    ['SELFIES'],
    ['CITYSCAPES', 'HOUSES', 'CITYSCAPES'],
    ['PEOPLE'],
    ['SPORT'],
    ['HOLIDAYS'],
    ['CRAFTS'],
    ['PERFORMANCES'],
    ['TRAVEL'],
    ['RECEIPTS','WEDDINGS','WHITEBOARDS','SCREENSHOTS','UTILITY','DOCUMENTS', 'MISC']
]

def writeToFile(f, mediaItems):
    for item in mediaItems:
        try:
            f.write('%s\n' %item['id'])
        except:
            print('Write error')

def get_hd_people_pics():
    print('Downloading HD pics...')
    try:
        res = requests.request("GET", GET_LIBRARIES_URL, headers=PHOTOS_HEADERS)
        res.json()
        albumID = res.json()['albums'][0]['id']
    except:
        print('Library request error') 
    
    payload = {
      "albumId": albumID,
      "pageSize": "5"
    }
    res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
    res = res.json()
    numPics = 0
    with open('picIDs.txt', 'w+') as f:
        while 'nextPageToken' in res and numPics < MAX_NUM_PICS:
            try:
                writeToFile(f, res['mediaItems'])
            except:
                print('HD GP media request error')
            numPics += len(res['mediaItems'])
            payload = {
              "albumId": albumID,
              "pageSize": "100",
              "pageToken": res['nextPageToken'],
            }
            res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
            res = res.json()
            
    f.close()
    print('Download complete!')
    return numPics

def get_month_pics(monthEntry):
    payload = {
      "filters": {
        "dateFilter": {
          "dates": [
            {
              "month": monthEntry['month'],
              "year": monthEntry['year']
            }
           ]
            }
          },
        "pageSize": "2"
      }
    
    res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
    res = res.json()
    numPics = 0
    with open('idFiles/months/'+monthEntry['name']+'PicIDs.txt', 'w+') as f:
        while 'nextPageToken' in res and numPics < MAX_NUM_PICS:
            try:
                writeToFile(f, res['mediaItems'])
            except:
                print('GP media request error with monthly filter')
            payload = {
              "filters": {
                "dateFilter": {
                  "dates": [
                    {
                      "month": monthEntry['month'],
                      "year": monthEntry['year']
                    }
                   ]
                }
              },
              "pageSize": "2",
              "pageToken": res['nextPageToken']
            }
            
            res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
            res = res.json()
            numPics += 2
    f.close()
    return numPics

def get_all_month_pics():
    print('Downloading pics with monthly filter...')
    for monthEntry in MONTHS:
        get_month_pics(monthEntry)
    print('Download complete!')
        
        
def get_category_pics(category):
    categoryLabel = category[0]
    if len(category)>1:
        categoryLabel = category[-1]
        category = category[0:len(category)-1]
    payload = {
      "filters": {
        "contentFilter": {
          "includedContentCategories": category
        }
      },
      "pageSize": "2"
    }
    
    res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
    res = res.json()
    numPics = 0
    with open('idFiles/categories/'+categoryLabel+'PicIDs.txt', 'w+') as f:
        while 'nextPageToken' in res and numPics < MAX_NUM_PICS:
            try:
                writeToFile(f, res['mediaItems'])
            except:
                print(res)
                print('GP media request error with category filter')
            payload = {
              "filters": {
                "contentFilter": {
                  "includedContentCategories": category
                }
              },
              "pageSize": "2",
              "pageToken": res['nextPageToken']
            }
            
            res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
            res = res.json()
            numPics += 2
    f.close()
    return numPics
    
def get_all_category_pics():
    print('Downloading pics with category filter...')
    for category in CATEGORIES:
        get_category_pics(category)
    print('Download complete!')

In [12]:
get_all_category_pics()

['ANIMALS,PETS', 'ANIMALS']
['FASHION']
['LANDMARKS']
['ARTS']
['FLOWERS', 'GARDENS', 'LANDSCAPES', 'NATURE']
['BIRTHDAYS']
['FOOD']
['NIGHT']
['SELFIES']
['CITYSCAPES']
['PEOPLE']
['SPORT']
['HOLIDAYS']
['CRAFTS']
['PERFORMANCES']
['TRAVEL']
['RECEIPTS', 'WEDDINGS', 'WHITEBOARDS', 'SCREENSHOTS', 'UTILITY', 'DOCUMENTS', 'HOUSES', 'MISC']


In [29]:
holidayID='AMyo5r0lZANxOJrBM7XH887PZfWTyK_x6LgX_n51XULlarUSqYiLm8g-L4xEEsX08zcy2HKmEDI8iMp09XTA0RCHYCR0MfxB7w'
res = requestIMG(holidayID)
downloadIMG(res['baseUrl'], 'holidayIMG.jpg')

Download IMG response token: <Response [200]>


## CompreFace Request

To request the CompreFace api, we need to first request the media item from the Google Photos API, which validates the baseUrl for 60 minutes, then download the image and then send this file in our request.

In [23]:
"""
    Request image with imgID
    We can only download the image from baseURL if we have requested
    the url from GP API and make our download request within 60 minutes
"""

def requestIMG(imgID):
    url = 'https://photoslibrary.googleapis.com/v1/mediaItems/'+imgID
    headers = {
        'content-type': 'application/json',
        'Authorization': 'Bearer {}'.format(creds.token)
    }
    res = requests.request("GET", url, headers=headers)
    return res.json()

"""
    Download image and place in WD
"""
def downloadIMG(url, file_name='imgToRecognize.jpg'):
    downloadResponse = requests.get(url)
    print('Download IMG response token:', downloadResponse)
    destination_folder = './downloads/'
    with open(os.path.join(destination_folder, file_name), 'wb') as f:
        f.write(downloadResponse.content)
        f.close()

"""
    Request the CompreFace API to recognize faces
"""
def recognizeFace(url):
    downloadIMG(url)
    headers = {
        'x-api-key': '0bedc62b-b2a4-4eb2-8efd-b62cc275e23c',
    }

    files = {
        'file': open('./downloads/imgToRecognize.jpg', 'rb'),
    }

    res = requests.post('http://localhost:8000/api/v1/recognition/recognize?face_plugins=landmarks, gender, age', headers=headers, files=files)
    print('Recognize face response token:', res)
    return res.json()

PHONES_TO_PERSON = {
    'BE2026': 'chimu',
    'SM-G970U':'shirleyWhirley',
    'iPhone 11': 'jiusus',
    'Pixel 3': 'bugBoy',
    'Pixel 5a': 'bugBoy',
    'iPhone 12': 'girlBoss',
    'iPhone X': 'me',
    'iPhone 8': 'girlBoss',
    'moto g(7) plus': 'chimu',
    'A6013': 'yuppie',
    'dumbestKid': 'iphone 7'
}

## Build matrices

Build the takerSubject, picturedWith, and month-based matrices. The values of each cell is a string of comma separated item IDs which we'll hopefully use to request pictures in our final visualizations

In [44]:
import pandas as pd
import numpy as np

names = ['me', 'girlBoss', 'bugBoy', 'jiusus', 'chimu', 'shirleyWhirley', 'yuppie', 'dumbestKid', 'emily']
RECOGNITION_THRESHOLD = .8 ## The similarity above which we allow a recognition

def getPictureTaker(GPRes):
    mediaMetadata = GPRes['mediaMetadata']
    if 'photo' in mediaMetadata:
        photo = mediaMetadata['photo']
        if 'cameraModel' in photo:
            phoneType = photo['cameraModel']
            try:
                return PHONES_TO_PERSON[phoneType]
            except:
                downloadIMG(GPres['baseUrl'], 'pictureTakerErr.jpg')
        else:
            return 'jiusus'
    else:
        return 'video'

"""
    Identify the faces in an image. Given the picture taker, increment the edge between
    pictureTaker and the face in the image.
    
"""
def processRecognition(res, pictureTaker, imgID, month=None):
    results = res['result']
    subjects = []
    for result in results: ## Iterates through every face in picture
        possibleSubjects = result['subjects']
        if len(possibleSubjects) == 0:
            continue
        else:
            if possibleSubjects[0]['similarity'] < RECOGNITION_THRESHOLD:
                continue
            else:
                photoSubject = possibleSubjects[0]['subject']
                subjects.append(photoSubject)
                takerSubjectMatrix.at[pictureTaker, photoSubject] += imgID+','
                if month is not None:
                    pictureOfSubjectByMonth.at[photoSubject, month] += imgID+','
    subject_i, subject_j = 0, 1
    while subject_i < len(subjects):
        firstSubject = subjects[subject_i]
        while subject_j < len(subjects):
            secondSubject = subjects[subject_j]
            picturedWithMatrix.at[firstSubject, secondSubject] += imgID+','
            picturedWithMatrix.at[secondSubject, firstSubject] += imgID+',' ## Make matrix symmetric for convenience
            subject_j += 1
        subject_i += 1
        subject_j = subject_i + 1

def createSubjectMatrices():
    print('Building subject-taker and photographed with matrices...')
    idFile = open('idFiles/picIDs.txt', 'r')
  
    for imgID in idFile.readlines():
        GPRes = requestIMG(imgID[:-1]) # Cut out the EOL token
        pictureTaker = getPictureTaker(GPRes)
        if pictureTaker == 'video':
            continue
        else:
            recognitionRes = recognizeFace(GPRes['baseUrl'])
            processRecognition(recognitionRes, pictureTaker, imgID)
    idFile.close()
    print('Matrices built!')
    
def createMonthMatrices():
    print('Building monthly matrices...')
    for monthEntry in MONTHS:
        idFile = open('idFiles/'+monthEntry['name']+'PicIDs.txt', 'r')
        for imgID in idFile.readlines():
            imgID = imgID[:-1]
            GPRes = requestIMG(imgID) # Cut out the EOL token
            pictureTaker = getPictureTaker(GPRes)
            if pictureTaker == 'video':
                 continue
            pictureBySubjectByMonth.at[pictureTaker, monthEntry['name']]+=imgID+','
            recognitionRes = recognizeFace(GPRes['baseUrl'])
            processRecognition(recognitionRes, pictureTaker, imgID, monthEntry['name'])
        idFile.close()
    print('Matrices built!')
        
        
def createCategoryMatrix():
    print('Building category matrix...')
    for category in CATEGORIES:
        idFile = open('idFiles/categories/'+category[-1]+'PicIDs.txt', 'r')
        for imgID in idFile.readlines():
            imgID = imgID[:-1] # Cut out the EOL token
            GPRes = requestIMG(imgID) 
            pictureTaker = getPictureTaker(GPRes)
            if pictureTaker == 'video':
                 continue
            subjectCategory.at[pictureTaker, category[-1]]+=imgID+','
        idFile.close()
    print('Matrix built!')

Run all our cells to populate the matrices for the frontend

In [49]:
takerSubjectMatrix = pd.DataFrame('', index=names, columns=names)
picturedWithMatrix = pd.DataFrame('', index=names, columns=names)
pictureBySubjectByMonth = pd.DataFrame('', index=names, columns=[monthEntry['name'] for monthEntry in MONTHS])
pictureOfSubjectByMonth = pd.DataFrame('', index=names, columns=[monthEntry['name'] for monthEntry in MONTHS])
subjectCategory = pd.DataFrame('', index=names, columns = [category[-1] for category in CATEGORIES])

#get_hd_people_pics()
#get_all_month_pics()
get_all_category_pics()

#createSubjectMatrices()
#createCategoryMatrix()
#createMonthMatrices()

Download complete!


Use the response of the API to write the results and required metadata into a data frame:

In [26]:
subjectCategory

,ANIMALS,FASHION,LANDMARKS,ARTS,NATURE,BIRTHDAYS,FOOD,NIGHT,SELFIES,CITYSCAPES,PEOPLE,SPORT,HOLIDAYS,CRAFTS,PERFORMANCES,TRAVEL,MISC
me,,,AMyo5r0H5ERsrqyA3w896t3xZVmtAsC1Bdee31B88st15W...,,,,AMyo5r0kPQ9qVk8B__Tu9KbPl_G0EAydi7n5r0VaRGyBHZ...,,AMyo5r0H5ERsrqyA3w896t3xZVmtAsC1Bdee31B88st15W...,AMyo5r0H5ERsrqyA3w896t3xZVmtAsC1Bdee31B88st15W...,AMyo5r0bhpa7r8C1aFmEbXK186dkgT97kAF8NBAv21XxiN...,AMyo5r2hOJFwHEFiD31Qb6-_EqusB88ZwQsAAYSC0S_KYU...,AMyo5r31W6k8Z_Wn6NU5Eqc8i2TvL-4JAXLbgXJGEq4A3I...,,,AMyo5r04316JtQC9dMd5J90jyqnYQJVmQ4_vrVKH4WCMjo...,
girlBoss,,,,,,,,,,,,,,,,,
bugBoy,,,,,AMyo5r1QarWfNj4jIDKOn8AbcB4OxY9aN3Y_7YJ0e4cRWd...,AMyo5r3ZLKXcbm0u39w4Bt9tyJhfBwl67mkF3yNZ9gjYqL...,,,,,,,,,,,
jiusus,,,,,,,,,,,,,,,,,
chimu,,,,,,,,,,,,,,,AMyo5r1vbLdwiYJLGrns78gBIjUo-2FLkmiZ34U5SmR-d7...,,
shirleyWhirley,,,,,,,,AMyo5r23ONwYXP672G-TT8JlQp3SwMWoMtnEUmBcNM9Se_...,,,,,,,,,AMyo5r23ONwYXP672G-TT8JlQp3SwMWoMtnEUmBcNM9Se_...
yuppie,,,,,,,,,,,,,,,,,
dumbestKid,,,,,,,,,,,,,,,,,
emily,,,,,,,,,,,,,,,,,


## Facial recognition "Authorization"